<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/Modelisation_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
#uploaded = files.upload()


import pandas as pd
from PIL import Image
from torchvision import transforms
import numpy as np

In [2]:
train_df_images = pd.read_json('train_df_images.json', orient="records", lines=True)

train_df_images

,item,name,description,type,category,subcategory,label,image
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,cultural exclusive,"[[[85, 93, 69], [64, 70, 47], [50, 55, 39], [1..."
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,cultural representative,"[[[33, 31, 17], [35, 33, 19], [32, 31, 19], [2..."
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,cultural representative,"[[[109, 110, 103], [113, 115, 107], [117, 118,..."
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,cultural representative,"[[[136, 91, 41], [123, 74, 26], [169, 134, 89]..."
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,cultural representative,"[[[106, 88, 76], [114, 96, 84], [115, 96, 85],..."
...,...,...,...,...,...,...,...,...
6246,http://www.wikidata.org/entity/Q321103,Bühl,"quarter of Tübingen, Baden-Württemberg, Germany",entity,geography,neighborhood,cultural exclusive,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
6247,http://www.wikidata.org/entity/Q338167,Tenderloin,area of New York City during the late 19th and...,entity,geography,neighborhood,cultural exclusive,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
6248,http://www.wikidata.org/entity/Q66991,Schinznach-Dorf,former municipality and current district of Sc...,entity,geography,neighborhood,cultural exclusive,"[[[61, 117, 223], [64, 118, 226], [65, 119, 22..."
6249,http://www.wikidata.org/entity/Q66922,Ependes,village and former municipality in Bois-d'Amon...,entity,geography,neighborhood,cultural exclusive,"[[[152, 146, 44, 254], [209, 198, 54, 255], [2..."


In [3]:
train_df_images['image'] = train_df_images['image'].apply(lambda x: np.array(x))

In [4]:
def resize_array(image):

    image = image.astype(np.uint8)
    img_pil = Image.fromarray(image)
    img_pil = img_pil.convert("RGB")
    img_pil = img_pil.resize((28, 28), Image.Resampling.LANCZOS)

    return np.array(img_pil)

In [5]:
train_df_images['image'] = train_df_images['image'].apply(lambda x: resize_array(x))

In [6]:
labels = train_df_images['label'].unique()
dict_mapping ={}

for i, label in enumerate(labels):
  dict_mapping[label]=int(i)
  i = i+1

print(dict_mapping)

train_df_images['label'] = train_df_images['label'].map(dict_mapping)
train_df_images['label'] = train_df_images['label'].apply(lambda x: np.array(x))

{'cultural exclusive': 0, 'cultural representative': 1, 'cultural agnostic': 2, 'cultural': 3, 'cult': 4, None: 5, 'cultural agn': 6, 'cultural represent': 7, 'cultural ex': 8, 'cultural ag': 9}


In [7]:
train_df_images.head()

,item,name,description,type,category,subcategory,label,image
0,http://www.wikidata.org/entity/Q306,Sebastián Piñera,Chilean entrepreneur and politician (1949–2024),entity,politics,politician,0,"[[[85, 93, 69], [64, 70, 47], [50, 55, 39], [1..."
1,http://www.wikidata.org/entity/Q12735,John Amos Comenius,"Czech teacher, educator, philosopher and write...",entity,politics,politician,1,"[[[33, 31, 17], [35, 33, 19], [32, 31, 19], [2..."
2,http://www.wikidata.org/entity/Q1752,Macrinus,Roman emperor from 217 to 218,entity,politics,politician,1,"[[[109, 110, 103], [113, 115, 107], [117, 118,..."
3,http://www.wikidata.org/entity/Q1639,Lamine Diack,Senegalese sports manager (1933–2021),entity,politics,politician,1,"[[[136, 91, 41], [123, 74, 26], [169, 134, 89]..."
4,http://www.wikidata.org/entity/Q9588,Richard Nixon,President of the United States from 1969 to 1974,entity,politics,politician,1,"[[[106, 88, 76], [114, 96, 84], [115, 96, 85],..."


Data augmentation, as we don't have a lot of data

In [16]:
#transform = transforms.Compose([
#    transforms.RandomResizedCrop(size=28, scale=(0.8, 1.0)),  # zoom entre 80% et 100%
#    transforms.RandomHorizontalFlip(p=0.5),
#    transforms.RandomRotation(degrees=15),
#    transforms.ToTensor()
#])

#nb_augmentation = 3

#for i in range(3):
#  train_df_images[f'augmented_image_{i}'] = None
#  train_df_images[f'augmented_image_{i}'] = train_df_images['image'].apply(lambda x: transform(Image.fromarray(x.astype(np.uint8))) if isinstance(x, np.ndarray) else None)

#train_df_images.head()

Modelisation

In [65]:
#for i in range(len(train_df_images)):
#  if train_df_images['image'][i].shape != (28, 28, 3):
#    print(train_df_images['image'][i].shape)

In [12]:
# Normalisation so that the fitting of the CNN is easier
X_train = np.stack(train_df_images['image'].to_numpy()).astype('float32') / 255.0
y_train = train_df_images['label']

# Checkings
print(X_train.shape)
print(X_train[0].shape)
print(type(X_train[0]))

print(y_train.shape)
print(y_train[0])

(6251, 28, 28, 3)
(28, 28, 3)
<class 'numpy.ndarray'>
(6251,)
0


In [13]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

model = models.Sequential([
    #Layer 1
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 3)),
    layers.MaxPooling2D((2, 2)),

    # Layer 2
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Layer 3
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])

# Compilation
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entraînement
model.fit(X_train, y_train, epochs= 40, batch_size=32)

Epoch 1/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.3032 - loss: nan
Epoch 2/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3123 - loss: nan
Epoch 3/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3053 - loss: nan
Epoch 4/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2977 - loss: nan
Epoch 5/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3013 - loss: nan
Epoch 6/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3034 - loss: nan
Epoch 7/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2943 - loss: nan
Epoch 8/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2998 - loss: nan
Epoch 9/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2999 - loss: nan
Epoch 10/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2991 - loss: nan
Epoch 11/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2992 - loss: nan
Epoch 12/40
196/196 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2965 - loss: nan
E